# Preprocessing

This pipeline is based on Nipype package, a 'node-based' python package for MRI preprocessing and analysis.  
We will use a pipeline that conssists of FSL and costume made interfaces (each is considered node).
This includes:
- BET (skull-stripping
- FAST (tissue segmentation
- FLIRT (Linear coregistration)
- FNIRT (Non-linear coregistration)
- ART (movement outlier detection)
- Isotropic Smoothing

Other nodes are 'vol_removal' for discarding first / last volumes, 'realign' for movement detection, and 'datasink' for output files.

In [3]:
# imports
from nipype import Node, Workflow, IdentityInterface, MapNode,SelectFiles, DataSink
from nipype.interfaces import fsl
from nipype.algorithms.rapidart import ArtifactDetect as ART

from PIL import Image
import numpy as np
import os

In [4]:
data = r"/home/anakin/Desktop/ds000113" # our prepared forrest dataset - make sure to change user!
output = r"/home/anakin/Desktop/ds000113/output/preprocessing" # where to save preprocess output

# we will again restrict our analysis to 8 subjects and two runs
num_of_subjects = 10 
num_of_runs = 2
excluded = [7,8]
subject_list = ["{:02d}".format(i) for i in range(1,num_of_subjects+1) if i not in excluded]

# locate your fsl folder to find these files
mni_template = "/usr/local/fsl/data/standard/MNI152_T1_2mm.nii.gz" 
mni_brain_template = "/usr/local/fsl/data/standard/MNI152_T1_2mm_brain.nii.gz"

# forrest dataset has a TR = 2 seconds
TR = 2

In [5]:
# infosurce node
infosource = Node(IdentityInterface(fields=["subject_list"]), name="infosource")
infosource.iterables = [('subject_list', subject_list)]

In [6]:
# selectfiles node
templates = {"anat": "sub-{subject_list}/ses-movie/anat/sub-{subject_list}_T1w.nii.gz",
             "func": "sub-{subject_list}/ses-movie/func/sub-{subject_list}_task-sync_run-*_bold.nii.gz",
             "conditions": "sub-{subject_list}/ses-movie/func/sub-{subject_list}_task-sync_run-*_events.tsv"}

selectfiles = Node(SelectFiles(templates),name="selectfiles")
selectfiles.inputs.base_directory = data

# datasink node
substitutions = [("_mean_img","mean_img"),
                 ("_subject_list",""),
                 ("_flirt_func","flirt_func"),
                 ("_func_warp","func_warp")]

substitutions += [("_{:02d}".format(i), "{:02d}".format(i)) for i in range(1,num_of_subjects+1) if i not in excluded]
substitutions += [("_art{:d}".format(i), "art_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("mean_img{:d}".format(i), "mean_img_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_realign{:d}".format(i), "realign_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("flirt_func{:d}".format(i), "flirt_func_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("func_warp{:d}".format(i), "func_warp_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_smooth{:d}".format(i), "smooth_run_{:d}".format(i+1)) for i in range(num_of_runs)]
substitutions += [("_vol_removal{:d}".format(i), "vol_removal_{:d}".format(i+1)) for i in range(num_of_runs)]

substitutions += [("seg_0", "CSF")]
substitutions += [("seg_1", "GM")]
substitutions += [("seg_2", "WM")]

datasink = Node(DataSink(base_directory=output),name="datasink")
datasink.inputs.substitutions = substitutions
datasink.output = output

In [7]:
# structural workflow 

bet = Node(fsl.BET(),name="bet")
bet.inputs.frac = 0.3 #value controlling how much of the skull to be removed - increasing might remove bits of brain
bet.inputs.vertical_gradient = 0.2 #value to linearly improve estimations at the bottom of the brain -increasing might understimate top
bet.inputs.mask = True #creates a binary mask of the brain

segment = Node(fsl.FAST(),name="segment")
"""You are interested mainly in the restored version of the structural image"""
segment.inputs.img_type = 1 # indicate its a T1 image
segment.inputs.number_classes = 3 # seperate structural into Grey Matter, White Matter and CSF
segment.inputs.output_biascorrected = True #return structural image bias field corrected
segment.inputs.output_biasfield = True #return bias field
segment.inputs.segments=True #create a binary mask for each tissue type
segment.inputs.no_pve=False #if you dont want partial volume estimation, set to true

flirt_struct = Node(fsl.FLIRT(),name="flirt_struct")
"""This step performs linear registration of the structural volume to the standard space"""
flirt_struct.inputs.reference = mni_brain_template
flirt_struct.inputs.dof = 12 # use 12 dof in structural flirt to prepare image for FNIRT nonlinear normalization to mni space
flirt_struct.inputs.out_matrix_file = "h2s_affine.mat" #keep this to facilite struct and func normalization to mni space using FNIRT
flirt_struct.inputs.interp = "spline" # change to trilinear if you are mostly interested in non-subcortical normalization. Trilinear is faster, but might have some rotational issues
flirt_struct.inputs.cost = "mutualinfo"

fnirt_struct = Node(fsl.FNIRT(),name="fnirt_struct")
"""This step performs non-linear registration of the structural volume to the standard space"""
fnirt_struct.inputs.ref_file = mni_template
fnirt_struct.inputs.config_file = "T1_2_MNI152_2mm"
fnirt_struct.inputs.fieldcoeff_file = True  #keep

In [8]:
# functional workflow

img_to_float = MapNode(fsl.ImageMaths(),name="img_to_float",iterfield=['in_file'])
"""This converts the 4d in float representation to faciliate some further computations"""
img_to_float.inputs.out_data_type='float'
img_to_float.inputs.op_string=''

vol_removal = MapNode(fsl.ExtractROI(), name="vol_removal",iterfield=['in_file'])
"""This step removes the 0 first volumes as segment edges have already been processed"""
vol_removal.inputs.t_min = 0 #number of scans to exclude from functional image
vol_removal.inputs.t_size = -1 # keep all the rest until the end

realign = MapNode(fsl.MCFLIRT(),name = "realign",iterfield=['in_file'])
"""This step realigns data to the first image"""
realign.inputs.save_mats = True #save realignment  parameters
realign.inputs.save_plots = True #save parameter plots
realign.inputs.dof = 6 #use this for rigid body correction

mean_img = MapNode(fsl.maths.MeanImage(),name= "mean_img",iterfield=['in_file'])
mean_img.inputs.dimension = "T" # find mean image across the time dimension

func_bet = MapNode(fsl.BET(),name= "func_bet",iterfield=['in_file'])
func_bet.inputs.frac = 0.5 #value controlling how much of the skull to be removed - increasing might remove bits of brain
func_bet.inputs.mask = True #creates a binary mask of the brain

art = MapNode(ART(),name="art",iterfield=["mask_file","realigned_files","realignment_parameters"])
art.inputs.mask_type="file"
art.inputs.norm_threshold= 3
art.inputs.zintensity_threshold = 3
art.inputs.parameter_source = "FSL"
art.inputs.use_norm=True
art.inputs.use_differences = [True, False]

flirt_func = MapNode(fsl.FLIRT(),name = "flirt_func",iterfield=['in_file'])
"""This step performs linear registration of the functional volume to the structural volume. It is mainly used to
extract the transformation matrix to later register to standard space"""
flirt_func.inputs.dof = 6
flirt_func.inputs.out_matrix_file = "f2h_affine.mat"
flirt_func.inputs.interp = "spline"
flirt_func.inputs.cost = "mutualinfo"

func_warp = MapNode(fsl.ApplyWarp(),name = "func_warp",
                    iterfield=['in_file',"premat"])
"""This step performs non linear registration to standard space using the affine matrix extracted
in the flirt above and the fnirt non linear warp in the structural pipeline """
func_warp.inputs.ref_file = mni_template

smooth = MapNode(fsl.IsotropicSmooth(),name="smooth",iterfield=['in_file'])
"""This step performs smoothing"""
smooth.inputs.fwhm = 6 #change the smoothing kernel

In [9]:
# creating workflow and connecting nodes

preprocessing = Workflow(name="preprocessing",base_dir=output)

preprocessing.connect(infosource,"subject_list",selectfiles, "subject_list")
preprocessing.connect(infosource,"subject_list",datasink, "container")                  
preprocessing.connect(selectfiles,"anat",bet, "in_file")  
preprocessing.connect(selectfiles,"anat",fnirt_struct, "in_file") 
                 
preprocessing.connect(bet,"out_file",segment,"in_files")
preprocessing.connect(segment,"restored_image",flirt_struct,"in_file")
preprocessing.connect(flirt_struct,"out_matrix_file",fnirt_struct,"affine_file")

preprocessing.connect(selectfiles,"func",img_to_float, "in_file")   
preprocessing.connect(img_to_float,"out_file",vol_removal,"in_file")
preprocessing.connect(vol_removal, "roi_file", realign, "in_file")
preprocessing.connect(realign, "out_file", mean_img, "in_file")

preprocessing.connect(mean_img, "out_file", func_bet, "in_file")
preprocessing.connect(func_bet, "mask_file", art, "mask_file")
preprocessing.connect(realign, "out_file", art, "realigned_files")
preprocessing.connect(realign, "par_file", art, "realignment_parameters")

preprocessing.connect(mean_img,"out_file",flirt_func,"in_file")
preprocessing.connect(realign,"out_file",func_warp,"in_file")
preprocessing.connect(flirt_func,"out_matrix_file",func_warp,"premat")
preprocessing.connect(segment,"restored_image",flirt_func,"reference")
preprocessing.connect(fnirt_struct,"fieldcoeff_file",func_warp,"field_file")
preprocessing.connect(func_warp,"out_file",smooth,"in_file")

In [8]:
# create preproc output graph
preprocessing.write_graph(graph2use='colored', format='png', simple_form=True,
                          dotfilename= os.path.join(output,"preprocessing_workflow.dot"))

# visualize the graph
img = Image.open(output + "/preprocessing_workflow.png")
img.show()

231010-22:08:57,100 nipype.workflow INFO:
	 Generated workflow graph: /home/anakin/Desktop/ds000113/output/preprocessing/preprocessing_workflow.png (graph2use=colored, simple_form=True).


In [9]:
preprocessing.connect(selectfiles,"conditions",datasink,"conditions")
preprocessing.connect(bet,"out_file",datasink,"bet.@bet_brain")
preprocessing.connect(bet,"mask_file",datasink,"bet.@bet_mask")
preprocessing.connect(segment,"restored_image",datasink,"segment.@restored_image")
preprocessing.connect(segment,"tissue_class_files",datasink,"segment.@tissue_class_files")
preprocessing.connect(segment,"bias_field",datasink,"segment.@bias_field")
preprocessing.connect(segment,"tissue_class_map",datasink,"segment.@tissue_class_map")
preprocessing.connect(flirt_struct,"out_file",datasink,"flirt_struct.@structural_flirt")
preprocessing.connect(flirt_struct,"out_matrix_file",datasink,"flirt_struct.@affine_parameters")
preprocessing.connect(fnirt_struct,"warped_file",datasink,"flirt_struct.@structural_fnirt")
preprocessing.connect(fnirt_struct,"field_file",datasink,"flirt_struct.@fieldcoeff_file")
preprocessing.connect(vol_removal,"roi_file",datasink,"vol_removal.@files")
preprocessing.connect(realign,"out_file",datasink,"realign.@realigned_files")
preprocessing.connect(realign,"par_file",datasink,"realign.@realig_parameters")
preprocessing.connect(art,"outlier_files",datasink,"art.@art_outliers")
preprocessing.connect(art,"plot_files",datasink,"art.@art_plot")
preprocessing.connect(mean_img,"out_file",datasink,"mean_img.@mean_img")
preprocessing.connect(flirt_func,"out_matrix_file",datasink,"flirt_func.@affine_parameters")
preprocessing.connect(flirt_func,"out_file",datasink,"flirt_func.@functional_flirt")
preprocessing.connect(func_warp,"out_file",datasink,"func_warp.@functional_fnirt")
preprocessing.connect(smooth,"out_file",datasink,"smooth.@smoothed_files")

my laptop did not handle paralelization of processes, but if you are working from a solid computer it is advised  
to change n_procs to 4 for faster preformance

In [10]:
# run preprocessing 
preprocessing.run(plugin='MultiProc', plugin_args={'n_procs' : 1})

231010-22:08:57,276 nipype.workflow INFO:
	 Workflow preprocessing settings: ['check', 'execution', 'logging', 'monitoring']
231010-22:08:57,379 nipype.workflow INFO:
	 Running in parallel.
231010-22:08:57,463 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 8 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231010-22:08:57,555 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.selectfiles" in "/home/anakin/Desktop/ds000113/output/preprocessing/preprocessing/_subject_list_01/selectfiles".
231010-22:08:57,560 nipype.workflow INFO:
	 [Node] Executing "selectfiles" <nipype.interfaces.io.SelectFiles>
231010-22:08:57,566 nipype.workflow INFO:
	 [Node] Finished "selectfiles", elapsed time 0.002481s.
231010-22:08:59,468 nipype.workflow INFO:
	 [Job 0] Completed (preprocessing.selectfiles).
231010-22:08:59,471 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 9 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231010-22:08:59,537 nipype.workf

231010-22:09:55,615 nipype.workflow INFO:
	 [Node] Finished "bet", elapsed time 10.039045s.
231010-22:09:57,526 nipype.workflow INFO:
	 [Job 14] Completed (preprocessing.bet).
231010-22:09:57,531 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 24 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231010-22:09:57,574 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.bet" in "/home/anakin/Desktop/ds000113/output/preprocessing/preprocessing/_subject_list_05/bet".
231010-22:09:57,579 nipype.workflow INFO:
	 [Node] Executing "bet" <nipype.interfaces.fsl.preprocess.BET>
231010-22:09:59,527 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 23 jobs ready. Free memory (GB): 13.37/13.57, Free processors: 0/1.
                     Currently running:
                       * preprocessing.bet
231010-22:10:08,23 nipype.workflow INFO:
	 [Node] Finished "bet", elapsed time 10.441573s.
231010-22:10:09,539 nipype.workflow INFO:
	 [Job 16] Completed (preprocessing.be

231010-23:30:02,394 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 23 jobs ready. Free memory (GB): 13.37/13.57, Free processors: 0/1.
                     Currently running:
                       * preprocessing.segment
231010-23:46:35,954 nipype.workflow INFO:
	 [Node] Finished "segment", elapsed time 995.505909s.
231010-23:46:37,393 nipype.workflow INFO:
	 [Job 34] Completed (preprocessing.segment).
231010-23:46:37,398 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 24 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231010-23:46:37,439 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.segment" in "/home/anakin/Desktop/ds000113/output/preprocessing/preprocessing/_subject_list_09/segment".
231010-23:46:37,448 nipype.workflow INFO:
	 [Node] Executing "segment" <nipype.interfaces.fsl.preprocess.FAST>
231010-23:46:39,395 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 23 jobs ready. Free memory (GB): 13.37/13.57, Free processors: 0/1.


231011-01:52:20,987 nipype.workflow INFO:
	 [Job 52] Completed (preprocessing.flirt_struct).
231011-01:52:20,992 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 24 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231011-01:52:21,38 nipype.workflow INFO:
	 [Node] Setting-up "preprocessing.flirt_struct" in "/home/anakin/Desktop/ds000113/output/preprocessing/preprocessing/_subject_list_10/flirt_struct".
231011-01:52:21,43 nipype.workflow INFO:
	 [Node] Executing "flirt_struct" <nipype.interfaces.fsl.preprocess.FLIRT>
231011-01:52:22,988 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 23 jobs ready. Free memory (GB): 13.37/13.57, Free processors: 0/1.
                     Currently running:
                       * preprocessing.flirt_struct
231011-02:06:53,75 nipype.workflow INFO:
	 [Node] Finished "flirt_struct", elapsed time 872.029358s.
231011-02:06:53,871 nipype.workflow INFO:
	 [Job 54] Completed (preprocessing.flirt_struct).
231011-02:06:53,877 nipy

231011-03:16:22,126 nipype.workflow INFO:
	 [Node] Setting-up "_img_to_float0" in "/home/anakin/Desktop/ds000113/output/preprocessing/preprocessing/_subject_list_01/img_to_float/mapflow/_img_to_float0".
231011-03:16:22,129 nipype.workflow INFO:
	 [Node] Executing "_img_to_float0" <nipype.interfaces.fsl.utils.ImageMaths>
231011-03:16:24,77 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 15 jobs ready. Free memory (GB): 13.37/13.57, Free processors: 0/1.
                     Currently running:
                       * _img_to_float0
231011-03:16:31,688 nipype.workflow INFO:
	 [Node] Finished "_img_to_float0", elapsed time 9.555732s.
231011-03:16:32,86 nipype.workflow INFO:
	 [Job 120] Completed (_img_to_float0).
231011-03:16:32,91 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 15 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231011-03:16:32,136 nipype.workflow INFO:
	 [Node] Setting-up "_img_to_float1" in "/home/anakin/Desktop/ds000113/output/preproc

...

231011-07:30:05,911 nipype.workflow INFO:
	 [Node] Finished "datasink", elapsed time 54.491719s.
231011-07:30:07,395 nipype.workflow INFO:
	 [Job 116] Completed (preprocessing.datasink).
231011-07:30:07,408 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 6 jobs ready. Free memory (GB): 13.57/13.57, Free processors: 1/1.
231011-07:30:07,454 nipype.workflow INFO:
	 [Node] Setting-up "_smooth0" in "/home/anakin/Desktop/ds000113/output/preprocessing/preprocessing/_subject_list_06/smooth/mapflow/_smooth0".
231011-07:30:07,458 nipype.workflow INFO:
	 [Node] Executing "_smooth0" <nipype.interfaces.fsl.maths.IsotropicSmooth>
231011-07:30:09,396 nipype.workflow INFO:
	 [MultiProc] Running 1 tasks, and 5 jobs ready. Free memory (GB): 13.37/13.57, Free processors: 0/1.
                     Currently running:
                       * _smooth0
231011-07:32:37,910 nipype.workflow INFO:
	 [Node] Finished "_smooth0", elapsed time 150.449603s.
231011-07:32:39,544 nipype.workflow INFO:
	 [Job 2

231011-07:36:02,708 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/06/mean_img/_subject_list_06/_mean_img0/sub-06_task-sync_run-1_bold_maths_roi_mcf_mean.nii.gz -> /home/anakin/Desktop/ds000113/output/preprocessing/06/mean_img/06/mean_img_run_1/sub-06_task-sync_run-1_bold_maths_roi_mcf_mean.nii.gz
231011-07:36:02,722 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/06/mean_img/_subject_list_06/_mean_img1/sub-06_task-sync_run-2_bold_maths_roi_mcf_mean.nii.gz -> /home/anakin/Desktop/ds000113/output/preprocessing/06/mean_img/06/mean_img_run_2/sub-06_task-sync_run-2_bold_maths_roi_mcf_mean.nii.gz
231011-07:36:02,735 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/06/art/_subject_list_06/_art0/art.sub-06_task-sync_run-1_bold_maths_roi_mcf_outliers.txt -> /home/anakin/Desktop/ds000113/output/preprocessing/06/art/06/art_run_1/art.sub-06_task-sync_run-1_bold_maths_roi_mcf_outliers.txt
231011-07:3

231011-07:42:38,312 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/09/flirt_struct/_subject_list_09/sub-09_T1w_warped.nii.gz -> /home/anakin/Desktop/ds000113/output/preprocessing/09/flirt_struct/09/sub-09_T1w_warped.nii.gz
231011-07:42:38,365 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/09/segment/_subject_list_09/sub-09_T1w_brain_restore.nii.gz -> /home/anakin/Desktop/ds000113/output/preprocessing/09/segment/09/sub-09_T1w_brain_restore.nii.gz
231011-07:42:38,528 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/09/segment/_subject_list_09/sub-09_T1w_brain_seg_0.nii.gz -> /home/anakin/Desktop/ds000113/output/preprocessing/09/segment/09/sub-09_T1w_brain_CSF.nii.gz
231011-07:42:38,539 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/09/segment/_subject_list_09/sub-09_T1w_brain_seg_1.nii.gz -> /home/anakin/Desktop/ds000113/output/preprocessing/09/segment/09/

231011-07:43:03,967 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/09/smooth/_subject_list_09/_smooth0/sub-09_task-sync_run-1_bold_maths_roi_mcf_warp_smooth.nii.gz -> /home/anakin/Desktop/ds000113/output/preprocessing/09/smooth/09/smooth_run_1/sub-09_task-sync_run-1_bold_maths_roi_mcf_warp_smooth.nii.gz
231011-07:43:14,448 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/09/smooth/_subject_list_09/_smooth1/sub-09_task-sync_run-2_bold_maths_roi_mcf_warp_smooth.nii.gz -> /home/anakin/Desktop/ds000113/output/preprocessing/09/smooth/09/smooth_run_2/sub-09_task-sync_run-2_bold_maths_roi_mcf_warp_smooth.nii.gz
231011-07:43:27,392 nipype.workflow INFO:
	 [Node] Finished "datasink", elapsed time 49.152621s.
231011-07:43:28,198 nipype.workflow INFO:
	 [Job 118] Completed (preprocessing.datasink).
231011-07:43:28,209 nipype.workflow INFO:
	 [MultiProc] Running 0 tasks, and 2 jobs ready. Free memory (GB): 13.57/13.57, Free processo

231011-07:49:30,19 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/10/realign/_subject_list_10/_realign0/sub-10_task-sync_run-1_bold_maths_roi_mcf.nii.gz.par -> /home/anakin/Desktop/ds000113/output/preprocessing/10/realign/10/realign_1/sub-10_task-sync_run-1_bold_maths_roi_mcf.nii.gz.par
231011-07:49:30,28 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/10/realign/_subject_list_10/_realign1/sub-10_task-sync_run-2_bold_maths_roi_mcf.nii.gz.par -> /home/anakin/Desktop/ds000113/output/preprocessing/10/realign/10/realign_2/sub-10_task-sync_run-2_bold_maths_roi_mcf.nii.gz.par
231011-07:49:30,31 nipype.interface INFO:
	 sub: /home/anakin/Desktop/ds000113/output/preprocessing/10/mean_img/_subject_list_10/_mean_img0/sub-10_task-sync_run-1_bold_maths_roi_mcf_mean.nii.gz -> /home/anakin/Desktop/ds000113/output/preprocessing/10/mean_img/10/mean_img_run_1/sub-10_task-sync_run-1_bold_maths_roi_mcf_mean.nii.gz
231011-07:49:30,42 nipyp

In [11]:
print('preprocessing is done!')

preprocessing is done!


In [13]:
# remove the working folder after data is in sink
import shutil
shutil.rmtree('/home/anakin/Desktop/ds000113/output/preprocessing/preprocessing')